# Motif Enrichment
- The motif hits matrix save the max loglikelyhood score of a motif in that DMR
- Before run the motif enrichment, I filter the scores for each motif using cutoff defined by
    - `np.quantile(motif_score[motif_score > 0]) * mask_quantile_to99`
- This filter will remove hits with low score, which is likely to be false positive
- Its likely boosting the significance of real enrichment

In [1]:
from concurrent.futures import ProcessPoolExecutor, as_completed
import anndata
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pybedtools
import scipy.stats as stats
import seaborn as sns
from scipy.sparse import csr_matrix
from statsmodels.stats.multitest import multipletests
import pathlib

In [2]:
mask_quantile_to99 = 0.9
or_cutoff = 1.5
neg_lgp_cutoff = 10
group = 'IT-L23+ACA'

motif_hits_path = '/home/hanliu/project/mouse_rostral_brain/DMR/ITSpatial_by_layer/IT-L23/MotifScan.h5ad'
dmr_hits_path = f'/home/hanliu/project/mouse_rostral_brain/DMR/ITSpatial_by_layer/IT-L23/HypoDMR/{group}.DMS1.bed'
output_dir = '/home/hanliu/project/mouse_rostral_brain/DMR/ITSpatial_by_layer/IT-L23/MotifEnrichment'


In [3]:
# Parameters
mask_quantile_to99 = 0.9
or_cutoff = 1.5
neg_lgp_cutoff = 10
group = "midlow"
motif_hits_path = "/home/hanliu/project/mouse_rostral_brain/DMR/DGmCHGroup/DMR/MotifScan.h5ad"
dmr_hits_path = "/home/hanliu/project/mouse_rostral_brain/DMR/DGmCHGroup/DMR/HypoDMR/midlow.DMS1.bed"
output_dir = "/home/hanliu/project/mouse_rostral_brain/DMR/DGmCHGroup/DMR/MotifEnrichment"


In [4]:
output_dir = pathlib.Path(output_dir)
output_dir.mkdir(exist_ok=True)

## DMR hits

In [5]:
dmr_bed_df = pd.read_csv(dmr_hits_path, sep='\t', header=None, index_col=3)
dmr_bed_df.head()

,0,1,2
3,,,
DGmCH_1,chr1,3069892,3069892
DGmCH_2,chr1,3084771,3084771
DGmCH_27,chr1,3759184,3759184
DGmCH_47,chr1,4335010,4335010
DGmCH_53,chr1,4412435,4412435


In [6]:
motif_hits = anndata.read_h5ad(motif_hits_path)
motif_hits = motif_hits[dmr_bed_df.index, :].copy()
motif_ids = motif_hits.var_names
motif_hits.X = motif_hits.X.todense()
motif_hits

AnnData object with n_obs × n_vars = 24024 × 719 
    obs: 'chrom', 'start', 'end'

In [7]:
# mask small motif scores
motif_cutoff = pd.Series(np.apply_along_axis(lambda i: np.quantile(i[i>0], 0.99) * mask_quantile_to99, 
                                             0, motif_hits.X),
                         index=motif_hits.var_names)
# only keep value larger than the cutoff for each motif
motif_hits.X = np.multiply(motif_hits.X, (motif_hits.X > motif_cutoff.values[None, :]))

## Background Hits

In [8]:
background_motif_hits = anndata.read_h5ad(
    '/home/hanliu/project/mouse_rostral_brain/DMR/BackgroundDMR/MotifScan.h5ad'
)

In [9]:
dmr_bed = pybedtools.BedTool().from_dataframe(dmr_bed_df)
bg_bed = pybedtools.BedTool().from_dataframe(
    background_motif_hits.obs.reset_index().iloc[:, [1, 2, 3, 0]])

In [10]:
# exclude background that overlap with DMR
bg_no_overlap = bg_bed.intersect(dmr_bed, v=True)
use_bg = bg_no_overlap.to_dataframe().iloc[:, -1].values
background_motif_hits = background_motif_hits[use_bg, :]

# make sure col in same order
background_motif_hits = background_motif_hits[:, motif_ids].copy()

In [11]:
judge_matrix = csr_matrix(background_motif_hits.X > motif_cutoff.values[None, :])
background_motif_hits.X = background_motif_hits.X.multiply(judge_matrix)

In [12]:
background_motif_hits

AnnData object with n_obs × n_vars = 339012 × 719 
    obs: 'chrom', 'start', 'end'

## Motif hits contingency table

In [13]:
# calculate motif occurence, not considering hits here
pos = (motif_hits.X > 0).sum(axis=0)
pos_total = motif_hits.shape[0]

neg = (background_motif_hits.X > 0).sum(axis=0)
neg_total = background_motif_hits.shape[0]

In [14]:
tables = {}
for motif, _pos, _neg in zip(motif_ids, pos.A1, neg.A1):
    table = [[_pos, pos_total - _pos], [_neg, neg_total - _neg]]
    tables[motif] = table

In [15]:
results = {}
with ProcessPoolExecutor(20) as executor:
    fs = {}
    for motif, t in tables.items():
        f = executor.submit(stats.fisher_exact, t, alternative='greater')
        fs[f] = motif

    for f in as_completed(fs):
        motif = fs[f]
        odds, p = f.result()
        results[motif] = {'oddsratio': odds, 'p_value': p}
motif_enrich_df = pd.DataFrame(results).T

_, p, _, _ = multipletests(motif_enrich_df['p_value'], method='fdr_bh')
motif_enrich_df['adj_p'] = p

motif_enrich_df['-lgp'] = -np.log10(motif_enrich_df['adj_p']).replace(
    -np.inf, -300)

records = {}
for motif, t in tables.items():
    tp, tn = t[0]
    fp, fn = t[1]
    tp_rate = tp / pos_total
    fp_rate = fp / neg_total
    records[motif] = dict(tp=tp,
                          tn=tn,
                          fp=fp,
                          fn=fn,
                          tp_rate=tp_rate,
                          fp_rate=fp_rate)
counts = pd.DataFrame(records).T
motif_enrich_df = pd.concat([motif_enrich_df, counts], axis=1, sort=True)

In [16]:
motif_enrich_df['Group'] = group

In [17]:
# final filter
filtered_motif_df = motif_enrich_df[(motif_enrich_df['oddsratio'] > or_cutoff)
                                    &
                                    (motif_enrich_df['-lgp'] > neg_lgp_cutoff)]
filtered_motif_df.shape[0]

9

In [18]:
motif_enrich_df.to_msgpack(output_dir / f'{group}.motif_enrichment.msg')

/home/hanliu/miniconda3/envs/clustering/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: to_msgpack is deprecated and will be removed in a future version.
It is recommended to use pyarrow for on-the-wire transmission of pandas objects.
  """Entry point for launching an IPython kernel.


In [19]:
pybedtools.cleanup()

In [20]:
filtered_motif_df

,oddsratio,p_value,adj_p,-lgp,tp,tn,fp,fn,tp_rate,fp_rate,Group
MA0507.1,1.839675,2.139585e-23,6.409839e-22,21.193153,353.0,23671.0,2726.0,336286.0,0.014694,0.008041,midlow
MA0627.2,1.780900,5.061494e-16,9.576880e-15,14.018776,256.0,23768.0,2038.0,336974.0,0.010656,0.006012,midlow
MA0785.1,1.963383,6.857115e-20,1.540708e-18,17.812280,246.0,23778.0,1777.0,337235.0,0.010240,0.005242,midlow
MA0786.1,1.838052,1.851856e-17,3.804241e-16,15.419732,256.0,23768.0,1975.0,337037.0,0.010656,0.005826,midlow
MA0787.1,1.879397,4.224661e-12,6.075062e-11,10.216449,158.0,23866.0,1190.0,337822.0,0.006577,0.003510,midlow
MA0788.1,2.183836,6.895208e-14,1.101701e-12,11.957936,127.0,23897.0,823.0,338189.0,0.005286,0.002428,midlow
MA0789.1,1.661894,2.471381e-23,7.107692e-22,21.148271,492.0,23532.0,4212.0,334800.0,0.020480,0.012424,midlow
MA0791.1,2.215518,5.408856e-13,8.454278e-12,11.072923,114.0,23910.0,728.0,338284.0,0.004745,0.002147,midlow
MA1115.1,1.534069,1.827228e-25,6.914615e-24,23.160232,748.0,23276.0,6956.0,332056.0,0.031136,0.020518,midlow
